In [2]:
import pandas as pd

municipios = pd.read_excel('resources/elecciones.xlsx', index_col=False)

In [6]:
# municipios.head()
municipios.columns = ['Nombre de Comunidad', 'Código de Provincia', 'Nombre de Provincia',
'Código de Municipio', 'Nombre de Municipio', 'Población',
'Número de mesas', 'Total censo electoral', 'Total votantes',
'Votos válidos', 'Votos a candidaturas', 'Votos en blanco',
'Votos nulos',
'PSOE',
'PP',
'Cs',
'PODEMOS-IU-EQUO',
'VOX',
'ERC-SOBIRANISTES',
'ECP-GUANYEM EL CANVI',
'JxCAT-JUNTS',
'EAJ-PNV',
'EH Bildu',
'PODEMOS-EU-MAREAS EN COMÚN-EQUO',
'CCa-PNC',
'NA+',
'COMPROMÍS 2019',
'PRC',
'PACMA',
'FRONT REPUBLICÀ',
'BNG',
'RECORTES CERO-GV',
'NCa',
'PACT',
'ARA-MES-ESQUERRA',
'GBAI',
'PUM+J',
'EN MAREA',
'PCTE',
'EL PI',
'AxSI',
'PCOE',
'PCPE',
'AVANT ADELANTE LOS VERDES',
'EB',
'CpM',
'SOMOS REGIÓN',
'PCPA',
'PH',
'UIG-SOM-CUIDES',
'ERPV',
'IZQP',
'PCPC',
'AHORA CANARIAS',
'CxG',
'PPSO',
'CNV',
'PREPAL',
'C.Ex-C.R.Ex-P.R.Ex',
'PR+',
'P-LIB',
'CILU-LINARES',
'ANDECHA ASTUR',
'JF',
'PYLN',
'FIA',
'FE de las JONS',
'F8',
'SOLIDARIA',
'DPL',
'UNIÓN REGIONALISTA',
'centrados',
'VOU',
'DP',
'PDSJE-UDEC',
'IZAR',
'RISA',
'C 21',
'+MAS+',
'UDT']


In [8]:
municipios.head()


,Nombre de Comunidad,Código de Provincia,Nombre de Provincia,Código de Municipio,Nombre de Municipio,Población,Número de mesas,Total censo electoral,Total votantes,Votos válidos,...,UNIÓN REGIONALISTA,centrados,VOU,DP,PDSJE-UDEC,IZAR,RISA,C 21,+MAS+,UDT
0,Andalucía,4,Almería,1,Abla ...,1235,2,1014,768,763,...,0,0,0,0,0,0,0,0,0,0
1,Andalucía,4,Almería,2,Abrucena ...,1150,2,1039,798,792,...,0,0,0,0,0,0,0,0,0,0
2,Andalucía,4,Almería,3,Adra ...,24859,29,17540,12174,12051,...,0,0,0,0,0,0,0,0,0,0
3,Andalucía,4,Almería,4,Albánchez ...,753,1,400,320,315,...,0,0,0,0,0,0,0,0,0,0
4,Andalucía,4,Almería,5,Alboloduy ...,621,1,533,448,442,...,0,0,0,0,0,0,0,0,0,0
